In [1]:
import requests
import pandas as pd

In [2]:
API_KEY = "579b464db66ec23bdd000001ba7c83c075784fde7ead526137a405e4"
url = "https://api.data.gov.in/resource/3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69"

In [3]:
params = {
    "api-key": API_KEY,
    "format": "json",
    "filters[city]": "Delhi",
    "limit": 100
}

In [4]:
response = requests.get(url, params=params)
data = response.json()
records = data["records"]
print(records[0])

{'country': 'India', 'state': 'Delhi', 'city': 'Delhi', 'station': 'Alipur, Delhi - DPCC', 'last_update': '15-01-2026 11:00:00', 'latitude': '28.815329', 'longitude': '77.153010', 'pollutant_id': 'CO', 'min_value': '49', 'max_value': '90', 'avg_value': '61'}


In [5]:
df = pd.DataFrame(records)
df = df[["pollutant_id", "avg_value"]]
df

,pollutant_id,avg_value
0,CO,61
1,PM10,266
2,NO2,37
3,CO,91
4,OZONE,19
...,...,...
95,PM10,274
96,CO,71
97,OZONE,8
98,NH3,9


In [6]:
pivot = df.pivot_table(
    values="avg_value",
    columns="pollutant_id",
    aggfunc="first"
)
pivot

pollutant_id,CO,NH3,NO2,OZONE,PM10,PM2.5,SO2
avg_value,61,17,37,19,266,377,7


In [7]:
pivot.columns = [c.lower().replace('.', '').replace(' ', '') for c in pivot.columns]
pivot = pivot.rename(columns={"pm25": "pm2_5"})
pivot

,co,nh3,no2,ozone,pm10,pm2_5,so2
avg_value,61,17,37,19,266,377,7


In [8]:
weather_url = (
    "https://api.open-meteo.com/v1/forecast?"
    "latitude=28.61&longitude=77.20&current="
    "temperature_2m,relative_humidity_2m,wind_speed_10m"
)

In [9]:
weather_data = requests.get(weather_url).json()
current = weather_data["current"]

In [10]:
pivot["temperature"] = current["temperature_2m"]
pivot["humidity"] = current["relative_humidity_2m"]
pivot["wind_speed"] = current["wind_speed_10m"]

In [11]:
pivot

,co,nh3,no2,ozone,pm10,pm2_5,so2,temperature,humidity,wind_speed
avg_value,61,17,37,19,266,377,7,13.2,60,6.6
